In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from konlpy.tag import Okt  
okt = Okt() 
import tensorflow as tf
import numpy as np
from collections import Counter
from konlpy.tag import Komoran
komoran = Komoran()
from wordcloud import WordCloud
import matplotlib.pyplot as plt


import urllib.request
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
import os
import json

In [2]:
import matplotlib.pyplot as plt 
%matplotlib inline
from string import punctuation
import warnings
warnings.filterwarnings('ignore')

In [3]:
emotion = pd.read_excel('한국어_단발성_대화_데이터셋.xlsx')
emotion

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
0,언니 동생으로 부르는게 맞는 일인가요..??,공포,NaN,NaN,NaN,놀람,5898.0
1,그냥 내 느낌일뿐겠지?,공포,NaN,NaN,NaN,분노,5665.0
2,아직너무초기라서 그런거죠?,공포,NaN,NaN,NaN,슬픔,5267.0
3,유치원버스 사고 낫다던데,공포,NaN,NaN,NaN,중립,4830.0
4,근데 원래이런거맞나요,공포,NaN,NaN,NaN,행복,6037.0
...,...,...,...,...,...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오,NaN,NaN,NaN,NaN,NaN
38590,재미가 없으니 망하지,혐오,NaN,NaN,NaN,NaN,NaN
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오,NaN,NaN,NaN,NaN,NaN
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오,NaN,NaN,NaN,NaN,NaN


In [4]:
emotion = emotion[['Sentence','Emotion']]

In [5]:
emotion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38594 entries, 0 to 38593
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  38594 non-null  object
 1   Emotion   38594 non-null  object
dtypes: object(2)
memory usage: 603.2+ KB


In [6]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
emotion

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포
...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오
38590,재미가 없으니 망하지,혐오
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오


In [8]:
train_data = emotion['Sentence']
train_data.head()

0    언니 동생으로 부르는게 맞는 일인가요..??
1                그냥 내 느낌일뿐겠지?
2              아직너무초기라서 그런거죠?
3               유치원버스 사고 낫다던데
4                 근데 원래이런거맞나요
Name: Sentence, dtype: object

In [9]:
train_emotion = emotion['Emotion']
train_emotion.head()

0    공포
1    공포
2    공포
3    공포
4    공포
Name: Emotion, dtype: object

In [10]:
train_emotion.unique()

array(['공포', '놀람', '분노', '슬픔', '중립', '행복', '혐오'], dtype=object)

In [113]:
emotion[emotion['Emotion']=='행복'].head(10)

,Sentence,Emotion
27128,유재석 오라버니 해피투게더 봤어요,행복
27129,우리모두 준혁이성과 LG선수들이 선전할수 있게 기를 넣어줍시다.,행복
27130,웃긴거 알죠?~ㅋㅋㅇ ㅏ~,행복
27131,덕분에 주방에서 요리하는게 넘 좋아졌어요,행복
27132,마지막 순간 손가락으로 총을 만들어 쏘실때에는 제가 직접 맞는 것 같습니다.,행복
27133,어떻게 여성 요리사 보다 더 인기가 많으신지...저도 늘 반찬 만들때 따라하고 있답니다.,행복
27134,우리에게 맛있는 음식을 전하시는 백주부님 당신 멋져,행복
27135,활동혜택으로 오션월드 입장권 2매까지 주는 폴인러버!,행복
27136,글구 지호 100일도 축하해요...ㅋㅋㅋㅋㅋㅋㅋㅋㅋ,행복
27137,응원합니다.그리고..사랑합니다. 화이팅!!!!,행복


In [11]:
emotion.to_csv('emotion_dataset.csv',index=False)

## scikit-learn 나이브 베이즈 감정 분석

In [17]:
cv = CountVectorizer()
transformed_text = cv.fit_transform(train_data)

In [18]:
clf = MultinomialNB()
clf.fit(transformed_text, train_emotion)

MultinomialNB()

In [25]:
test_data = ['너를 만난 세상 슬픈 우연 속에서','잊혀지지 않는 너의 모습 그리며 우네', '나는 나는 행복에 묻힐 거에요', '눈을 뜨기 힘든 가을 보다 높은 저 하늘이 기분 좋아', '두 눈에 비친 너의 미소 지친 날 감싸듯']

In [26]:
doc_vector = cv.transform(test_data)
test_result = clf.predict(doc_vector)
print(test_result)

['슬픔' '행복' '슬픔' '행복' '행복']


---

## 감정별 특정 단어 출현 가능도

In [44]:
def cal_partial_freq(emotion_set, emotion):
    partial_freq = dict()
    filtered_texts = emotion_set[emotion_set['Emotion']==emotion]
    filtered_texts = filtered_texts['Sentence']
    
    # 전체 데이터 내 각 단어별 빈도수를 입력해 주는 부분
    for sent in filtered_texts:
        # words = sent.rstrip().split()
        # for word in words:
        for word in sent.rstrip().split():
            if word not in partial_freq:
                partial_freq[word] = 1
            else:
                partial_freq[word] += 1
    
    return partial_freq

In [46]:
def cal_total_freq(partial_freq):
    total = 0
    # partial_freq 딕셔너리에서 감정별로 문서 내 전체 단어의 빈도수를 계산하여 반환하는 부분 
    for word, freq in partial_freq.items():
        total += freq
    
    return total

In [47]:
emotion_set = emotion

In [62]:
# '좋아'라는 단어가 행복이라는 감정을 표현하는 문장에서 발생할 가능도 
good_counter = cal_partial_freq(emotion_set, "행복")
good_likelihood = good_counter['좋아'] / cal_total_freq(good_counter)
print(good_likelihood)

6.89464975179261e-05


In [108]:
# '사랑'이라는 단어가 행복이라는 감정을 표현하는 문장에서 발생할 가능도 
love_counter = cal_partial_freq(emotion_set, "행복")
love_likelihood = love_counter['사랑'] / cal_total_freq(love_counter)
print(love_likelihood)

0.00017236624379481522


In [54]:
# '슬퍼요'라는 단어가 슬픔이라는 감정을 표현하는 문장에서 발생할 가능도
sad_likelihood = sad_counter['슬퍼요'] / cal_total_freq(sad_counter)
print(sad_likelihood)

3.83670963781461e-05


In [55]:
sup_counter =  cal_partial_freq(emotion_set, "놀람")
sup_likelihood = sup_counter['신기'] / cal_total_freq(sup_counter)
print(sup_likelihood)

7.43080066877206e-05


---

In [123]:
from sklearn import datasets 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.naive_bayes import MultinomialNB

In [119]:
train_data.head()

0    언니 동생으로 부르는게 맞는 일인가요..??
1                그냥 내 느낌일뿐겠지?
2              아직너무초기라서 그런거죠?
3               유치원버스 사고 낫다던데
4                 근데 원래이런거맞나요
Name: Sentence, dtype: object

In [118]:
train_emotion.head()

0    공포
1    공포
2    공포
3    공포
4    공포
Name: Emotion, dtype: object

In [120]:
# 입력 데이터와 타깃 데이터
X, y = train_data, train_emotion

In [124]:
# 데이터 분리 ( train / test )
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 1)

In [126]:
# 데이터 확인
print(X_train[0], y_train[0])

언니 동생으로 부르는게 맞는 일인가요..?? 공포


## TF-IDF 벡터 변환

Tfidftransformer & Tfidfvectorizer
> https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.YZerCd8QuUk

* 서로 다른 작업에 대해 빈도(term count) 벡터가 필요하다면 Tfidftransformer를 사용
* 학습 (training) 데이터 세트 내의 문서에 대한 TF-idf 점수를 계산해야 하는 경우 Tfidfvectorizer를 사용
* 학습 (training) 데이터 세트 외부에 있는 문서에서 tf-idf 점수를 계산해야 하는 경우 둘 중 하나를 사용하면 모두 작동한다.

In [127]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB   #다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score     #정확도 계신

In [128]:
cv = CountVectorizer()
X_train_cv = cv.fit_transform(X_train)
print(X_train_cv.shape) # 전체 샘플의 개수와 단어의 수

(28945, 74954)


In [129]:
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(X_train_cv)
print(tfidfv.shape)

(28945, 74954)


In [130]:
# 다중 분류 나이브 베이즈 분류 모델 객체
model = MultinomialNB(alpha= 0.01)

In [131]:
model.fit(tfidfv, y_train)

MultinomialNB(alpha=0.01)

* alpha <br>
표본 데이터의 수가 적은 경우에는 베르누이 모수가 0 또는 1이라는 극단적인 모수 추정값이 나올 수도 있다. 하지만 현실적으로는 실제 모수값이 이런 극단적인 값이 나올 가능성이 적다. 따라서 베르누이 모수가 0.5인 가장 일반적인 경우를 가정하여 0이 나오는 경우와 1이 나오는 경우, 두 개의 가상 표본 데이터를 추가한다. 그러면 0이나 1과 같은 극단적인 추정값이 0.5에 가까운 다음과 값으로 변한다. 이를 라플라스 스무딩(Laplace smoothing) 또는 애드원(Add-One) 스무딩이라고 한다.

In [132]:
X_test_cv = cv.transform(X_test)
tfidfv_test = tfidf_transformer.transform(X_test_cv)

predicted = model.predict(tfidfv_test)

print("accuracy :", accuracy_score(y_test, predicted))

accuracy : 0.3558918022593015


In [133]:
# 학습 데이터 정확도
model.score(tfidfv, y_train)

0.9900846432889964

In [134]:
# 테스트 데이터 정확도
model.score(tfidfv_test, y_test)

0.3558918022593015